# Convolutional Neural Network

In [52]:
import os, random
import numpy as np

from os.path                    import join, basename
from scipy.misc                 import imresize, imread
from datetime                   import datetime
from keras.models               import Sequential
from keras.layers               import Dense, Activation, Flatten, Dropout
from keras.layers.pooling       import MaxPooling2D
from keras.layers.convolutional import Conv2D, ZeroPadding2D
from keras.optimizers           import Adam
from keras.layers.advanced_activations import LeakyReLU



In [53]:
def accuracy(model, X,  y):
    "Return the accuracy of model on Inputs X and labels y."
    y_hat = model.predict_classes(X, verbose=0)
    n_correct = (np.array(y_hat) == np.array(y)).sum()
    return n_correct / float(y.size)

In [54]:

def prod(z):
    "Return the product z[0] * z[1] * ... * z[-1]."
    result = 1
    for i in range(0, len(z)):
        result *= z[i]
    return result

In [55]:
def load_course_images(directory, image_size):
    labels = []
    files  = [f for f in os.listdir(directory) if f.endswith("jpg")]
    shape  = [len(files)] + [image_size, image_size, 4]
    X      = np.zeros(shape = shape) + np.nan  # Put a NaN in each cell to make sure we 
    
    for imageIndex, f in enumerate(files):
        I = imread("{}/{}".format(directory, f))
        I = imresize(I, [image_size, image_size])
        X[imageIndex, :, :, 0:3] = I / 255.0                # rgb  channels
        X[imageIndex, :, :, 3]   = (I.mean(axis=2) / 255.0) # gray channel
        labels.append(f.split("_")[0]) # Pull the LABEL out of the image name
        
    assert np.isnan(X).sum() == 0 # Make sure no NaNs remain in the data.
    y        = np.array([LABELS.index(lbl) for lbl in labels]) # Collect all the LABELs.
    y_onehot = np.zeros((X.shape[0], len(LABELS))) # Now need to convert to a 'one hot'
    for rowIndex, colIndex in enumerate(y):  # This will produce ( (0, y(0)), (1, y
        y_onehot[rowIndex, colIndex] = 1 # For each image, set one output unit to 1.  
    return X, y, y_onehot, [join(directory, f) for f in files]


In [56]:
IMG_DIR           = "/Users/gautam/IdeaProjects/Lab3DNN/images/"


In [57]:
LABELS            = ['airplanes', 'butterfly', 'flower', 'grand', 'starfish', 'watch']   
imageDimension    = 32
numberOfColors    =   4 # R, G, B, and Gray
epochsToRun       = 100
batch_size        =  10 # how many gradients we collect before updating weights
platesConv1       =  16
platesConv2       =  16
kernelSizeConv1   =   4
kernelSizePool1   =   2
kernelSizeConv2   =   4
kernelSizePool2   =   2
strideConv1       =   1 # same for both x and y dimensions
stridePool1       =   2 # same for both x and y dimensions
strideConv2       =   1 # same for both x and y dimensions
stridePool2       =   2 # same for both x and y dimensions
zeroPaddingConv1  =   1 # same for both x and y dimensions (this is padding of the INPUT 
zeroPaddingPool1  =   1 # same for both x and y dimensions (this is padding of the CONV1
zeroPaddingConv2  =   1 # same for both x and y dimensions (this is padding of the POOL1 
zeroPaddingPool2  =   1 # same for both x and y dimensions (this is padding of the CONV2
input_dropoutProb =   0.05
conv1_dropoutProb =   0.50
pool1_dropoutProb =   0.00
conv2_dropoutProb =   0.50
pool2_dropoutProb =   0.00
final_dropoutProb =   0.50
numberOfFinalHUs  = 128
numberOfClasses   = len(LABELS)

In [58]:
confusionTestsetAtBestTuneset = np.zeros((numberOfClasses, numberOfClasses))
bestTuneSetEpoch     = np.nan
bestTuneSetAcc       = 0
testSetAccAtBestTune = 0


X_train, y_train, y_onehot_train, img_files_train = load_course_images(directory="{}/trainset".format(IMG_DIR), image_size=imageDimension)
X_tune,  y_tune,  y_onehot_tune,  img_files_tune  = load_course_images(directory="{}/tuneset".format( IMG_DIR), image_size=imageDimension)
X_test,  y_test,  y_onehot_test,  img_files_test  = load_course_images(directory="{}/testset".format( IMG_DIR), image_size=imageDimension)
print("There are {:,} training examples.".format(len(X_train)))
print("There are {:,} tuning examples.".format(  len(X_tune)))
print("There are {:,} testing examples.".format( len(X_test)))

There are 554 training examples.
There are 180 tuning examples.
There are 178 testing examples.


In [59]:
model = Sequential()
leakyReLUtoUse = LeakyReLU(alpha = 0.1)
model.add(Conv2D(platesConv1, kernel_size = kernelSizeConv1,
                 input_shape = [imageDimension, imageDimension, numberOfColors],data_format = "channels_last", strides     = strideConv1, 
                 padding     = "valid", 
                 use_bias    = True))
                 
model.add(leakyReLUtoUse); 
model.add(ZeroPadding2D(padding = zeroPaddingConv1, data_format = "channels_last"))
model.add(Dropout(conv1_dropoutProb)) 

model.add(MaxPooling2D(pool_size = kernelSizePool1, strides = stridePool1, padding = 'valid'))
model.add(Dropout(pool1_dropoutProb))
model.add(ZeroPadding2D(padding  = zeroPaddingPool1))

model.add(Conv2D(platesConv2, 
                 kernel_size = kernelSizeConv2,
                 strides     = strideConv2, 
                 padding     = "valid", 
                 use_bias    = True))
model.add(leakyReLUtoUse); # Have to add as a layer, not as an argument to Conv2D.
model.add(Dropout(conv2_dropoutProb))
model.add(ZeroPadding2D(padding=  zeroPaddingConv2))

model.add(MaxPooling2D(pool_size = kernelSizePool2, strides = stridePool2, padding = 'valid'))
model.add(Dropout(pool2_dropoutProb))
model.add(ZeroPadding2D(padding  = zeroPaddingPool2))

model.add(Flatten()) 
model.add(Dense(units = numberOfFinalHUs))
model.add(Activation('relu'))
model.add(Dropout(final_dropoutProb))

model.add(Dense(units = numberOfClasses))
model.add(Activation("softmax"))

In [60]:
numberOfWeights = 0
for tw in model.trainable_weights:
    numberOfWeights += prod([dim.value for dim in tw.get_shape()])
print()
print("The model has {:,} trainable weights.".format(numberOfWeights))
print()

optimizerToUse = Adam() 
model.compile(loss = 'categorical_crossentropy', optimizer = optimizerToUse, metrics = ['accuracy']) 
for i in range(epochsToRun):
    model.fit(X_train, 
              y_onehot_train, 
              epochs     = 1, 
              batch_size = batch_size,
              verbose    = 0,  shuffle    = True)
              
    acc_train = accuracy(model, X_train, y_train)
    acc_tune  = accuracy(model, X_tune, y_tune)
    acc_test  = accuracy(model, X_test, y_test)
    
    marker    = " "
    if acc_tune > bestTuneSetAcc:
        bestTuneSetEpoch     = i
        bestTuneSetAcc       = acc_tune
        testSetAccAtBestTune = acc_test
        marker               = "*"
        y_pred_test = model.predict_classes(X_test, verbose = 0)
        confusionTestsetAtBestTuneset.fill(0)
        for y_pred, y in zip(y_pred_test, y_test):
            confusionTestsetAtBestTuneset[y, y_pred] += 1

    print("After {:>4} epochs, accuracies are:  train = {:.3f}  tune = {:.3f}{} test = {:.3f}    {}".format(i+1, acc_train, acc_tune, marker, acc_test, datetime.now().strftime('%H:%M:%S  %m-%d-%Y')))
    
# Done with training.
print()
print("Final accuracy train = {:.3f}, tune = {:.3f}, test = {:.3f}".format(acc_train, acc_tune, acc_test))


    
print()



The model has 210,854 trainable weights.

After    1 epochs, accuracies are:  train = 0.556  tune = 0.522* test = 0.562    06:49:57  04-01-2017
After    2 epochs, accuracies are:  train = 0.605  tune = 0.600* test = 0.624    06:49:58  04-01-2017
After    3 epochs, accuracies are:  train = 0.670  tune = 0.656* test = 0.646    06:50:00  04-01-2017
After    4 epochs, accuracies are:  train = 0.776  tune = 0.683* test = 0.680    06:50:02  04-01-2017
After    5 epochs, accuracies are:  train = 0.827  tune = 0.689* test = 0.713    06:50:04  04-01-2017
After    6 epochs, accuracies are:  train = 0.877  tune = 0.750* test = 0.742    06:50:06  04-01-2017
After    7 epochs, accuracies are:  train = 0.892  tune = 0.750  test = 0.742    06:50:09  04-01-2017
After    8 epochs, accuracies are:  train = 0.910  tune = 0.761* test = 0.747    06:50:12  04-01-2017
After    9 epochs, accuracies are:  train = 0.915  tune = 0.778* test = 0.764    06:50:14  04-01-2017
After   10 epochs, accuracies are:  tra

In [61]:
print("Best testset accuracy at Epoch #{} = {:.3f}".format(bestTuneSetEpoch+1, testSetAccAtBestTune))
print()
print("Testset confusionTestsetAtBestTuneset matrix chosen by early stopping (rows are true class, cols are predicted class)")
print()
fmt = " | {:>10} {:>10} {:>10} {:>10} {:>10} {:>10}"
print("{:10}".format("") + fmt.format(*LABELS))
print("------------------------------------------------------------------------------")

for rowIndex in range(numberOfClasses):
    print("{:10}".format(LABELS[rowIndex]) + fmt.format(*confusionTestsetAtBestTuneset[rowIndex, :]))

Best testset accuracy at Epoch #36 = 0.820

Testset confusionTestsetAtBestTuneset matrix chosen by early stopping (rows are true class, cols are predicted class)

           |  airplanes  butterfly     flower      grand   starfish      watch
------------------------------------------------------------------------------
airplanes  |       39.0        0.0        1.0        0.0        0.0        1.0
butterfly  |        2.0       10.0        1.0        0.0        3.0        2.0
flower     |        0.0        1.0       31.0        1.0        3.0        1.0
grand      |        1.0        1.0        1.0       16.0        0.0        0.0
starfish   |        1.0        2.0        2.0        0.0        9.0        3.0
watch      |        0.0        2.0        2.0        0.0        1.0       41.0


In [62]:
### Create a web page to view the errors.
with open(join(IMG_DIR, "errors.html"), "w") as F:
    F.write("<table> \n".format(LABELS[y_pred]))
    for y_pred, y, filename in zip(y_pred_test, y_test, img_files_test): # See zip example above.
        if y_pred != y:
            img_file = join("testset", basename(filename))
            F.write("<tr> <td>I thought this was a {}</td>\n".format(LABELS[y_pred]))
            F.write("<td> <img src='{}' style='border-color: red' border=5> </td>\n".format(img_file))
            F.write("</tr>\n")